In [5]:
import os
import cv2
import numpy as np
import random
from math import pi, cos, sin, radians

# --- パラメータ一括管理 ---
CONFIG = {
    "img_size": 512,                 # 画像の幅・高さ（正方形）
    "num_images": 100,               # 生成する画像の枚数
    "max_shapes_per_image": 5,       # 1画像あたりの最大図形数
    "jitter_point": 3,               # 頂点座標を揺らす範囲（px単位）
    "vertex_jitter_triangle_square": 5,  # 三角形・四角形の頂点揺らし幅
    "circle_points_num": 30,         # 円を構成する点の数（多いほど滑らか）
    "circle_radius_range": (20, 80),# 円の半径の最小・最大値
    "polygon_size_range": (40, 100),# 三角形・四角形のサイズ範囲
    "rotation_angle_range": (-15, 15), # 図形を回転させる角度の範囲（度数法）
    "line_thickness_range": (2, 4), # 通常線の太さ範囲（ランダム）
    "normal_line_color": 255,        # 通常の線の色（白）
}

# 保存ディレクトリ作成
os.makedirs("images", exist_ok=True)
os.makedirs("labels", exist_ok=True)

def jitter_point(x, y, jitter):
    """
    指定した揺らし幅で座標をランダムにずらす
    """
    return x + random.uniform(-jitter, jitter), y + random.uniform(-jitter, jitter)

def rotate_points(pts, angle_deg, center):
    """
    指定中心(center)を軸に点群ptsをangle_deg度回転させる
    """
    angle_rad = radians(angle_deg)
    cos_a = cos(angle_rad)
    sin_a = sin(angle_rad)
    cx, cy = center
    new_pts = []
    for (x, y) in pts:
        tx, ty = x - cx, y - cy
        rx = tx * cos_a - ty * sin_a + cx
        ry = tx * sin_a + ty * cos_a + cy
        new_pts.append((rx, ry))
    return new_pts

def draw_handwritten_polygon(img, pts_np):
    """
    手書き風の多角形を描画する補助関数（途切れ線なし）
    ・通常線だけ描画する
    """
    thickness_normal = random.randint(*CONFIG["line_thickness_range"])
    color_normal = CONFIG["normal_line_color"]

    # 閉じた線を描画（通常線のみ）
    cv2.polylines(img, [pts_np], isClosed=True, color=color_normal, thickness=thickness_normal)

def draw_handwritten_circle(img):
    """
    手書き風の円を描く関数
    ・円周を複数点で近似し、点ごとに座標を揺らす
    ・回転を加える
    ・太さランダムの閉じたポリラインで描画
    ・バウンディングボックス・面積を計算して返す
    """
    r = random.randint(*CONFIG["circle_radius_range"])
    cx = random.randint(r, CONFIG["img_size"] - r)
    cy = random.randint(r, CONFIG["img_size"] - r)
    num_points = CONFIG["circle_points_num"]
    angle_step = 360 / num_points
    pts = []
    for i in range(num_points):
        angle = angle_step * i
        rad = radians(angle)
        radius_jitter = r + random.uniform(-3, 3)  # 半径を微妙に変える
        x = cx + radius_jitter * cos(rad)
        y = cy + radius_jitter * sin(rad)
        x, y = jitter_point(x, y, CONFIG["jitter_point"])
        pts.append((x, y))

    angle_rot = random.uniform(*CONFIG["rotation_angle_range"])
    pts = rotate_points(pts, angle_rot, (cx, cy))

    pts_np = np.array(pts, np.int32)
    thickness = random.randint(1, 3)

    cv2.polylines(img, [pts_np], isClosed=True, color=CONFIG["normal_line_color"], thickness=thickness)

    x, y, w, h = cv2.boundingRect(pts_np)
    area = pi * (r ** 2)  # 面積は理論値で計算

    return 0, [x, y, w, h], area

def draw_handwritten_triangle(img):
    """
    手書き風の三角形を描く関数
    ・頂点座標を揺らし回転も加える
    ・多角形描画を呼ぶ（途切れ線なし）
    ・バウンディングボックス・面積を計算して返す
    """
    size = random.randint(*CONFIG["polygon_size_range"])
    x = random.randint(0, CONFIG["img_size"] - size)
    y = random.randint(0, CONFIG["img_size"] - size)

    pts = [
        (x, y + size),
        (x + size / 2, y),
        (x + size, y + size)
    ]
    pts = [jitter_point(px, py, CONFIG["vertex_jitter_triangle_square"]) for (px, py) in pts]

    cx = sum(p[0] for p in pts) / 3
    cy = sum(p[1] for p in pts) / 3
    angle_rot = random.uniform(*CONFIG["rotation_angle_range"])
    pts = rotate_points(pts, angle_rot, (cx, cy))

    pts_np = np.array(pts, np.int32)

    draw_handwritten_polygon(img, pts_np)

    x_b, y_b, w, h = cv2.boundingRect(pts_np)
    x1, y1 = pts[0]
    x2, y2 = pts[1]
    x3, y3 = pts[2]
    # 三角形の面積公式（外積の半分）
    area = abs((x1*(y2 - y3) + x2*(y3 - y1) + x3*(y1 - y2)) / 2)

    return 1, [x_b, y_b, w, h], area

def draw_handwritten_square(img):
    """
    手書き風の四角形を描く関数
    ・頂点座標を揺らし回転も加える
    ・多角形描画を呼ぶ（途切れ線なし）
    ・バウンディングボックス・面積を計算して返す
    """
    size = random.randint(*CONFIG["polygon_size_range"])
    x = random.randint(0, CONFIG["img_size"] - size)
    y = random.randint(0, CONFIG["img_size"] - size)

    pts = [
        (x, y),
        (x + size, y),
        (x + size, y + size),
        (x, y + size)
    ]
    pts = [jitter_point(px, py, CONFIG["vertex_jitter_triangle_square"]) for (px, py) in pts]

    cx = sum(p[0] for p in pts) / 4
    cy = sum(p[1] for p in pts) / 4
    angle_rot = random.uniform(*CONFIG["rotation_angle_range"])
    pts = rotate_points(pts, angle_rot, (cx, cy))

    pts_np = np.array(pts, np.int32)

    draw_handwritten_polygon(img, pts_np)

    x_b, y_b, w, h = cv2.boundingRect(pts_np)
    area = w * h  # 四角形の面積は矩形面積で近似

    return 2, [x_b, y_b, w, h], area

def normalize_bbox(bbox):
    """
    YOLO形式用にバウンディングボックスを正規化
    (中心x, 中心y, 幅, 高さ) / 画像サイズで[0,1]に収める
    """
    x, y, w, h = bbox
    cx = (x + w / 2) / CONFIG["img_size"]
    cy = (y + h / 2) / CONFIG["img_size"]
    nw = w / CONFIG["img_size"]
    nh = h / CONFIG["img_size"]
    return cx, cy, nw, nh

draw_funcs = [draw_handwritten_circle, draw_handwritten_triangle, draw_handwritten_square]

for i in range(CONFIG["num_images"]):
    # 黒背景画像を作成
    img = np.zeros((CONFIG["img_size"], CONFIG["img_size"]), dtype=np.uint8)
    label_lines = []
    num_shapes = random.randint(1, CONFIG["max_shapes_per_image"])
    for _ in range(num_shapes):
        # ランダムに図形を選んで描画
        shape_fn = random.choice(draw_funcs)
        class_id, bbox, area = shape_fn(img)

        # バウンディングボックスの正規化
        cx, cy, w, h = normalize_bbox(bbox)

        # 面積は画像全体に対する割合で正規化
        area_norm = area / (CONFIG["img_size"] * CONFIG["img_size"])

        # ラベル行に書式化して追加
        label_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f} {area_norm:.6f}")

    # 画像ファイルとラベルファイルを書き出し
    cv2.imwrite(f"images/img_{i:04d}.png", img)
    with open(f"labels/img_{i:04d}.txt", "w") as f:
        f.write("\n".join(label_lines))

print(f"{CONFIG['num_images']}枚の手書き風図形画像とラベルを生成しました！")


100枚の手書き風図形画像とラベルを生成しました！
